In [1]:
import numpy as np
import pandas as pd
import os
import requests
import io
from datetime import datetime, timedelta
import multiprocessing
import arviz as az
import logging
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error
import xgboost as xgb
from scipy.stats import poisson
import sqlite3

In [5]:
conn = sqlite3.connect(r"C:\Users\Owner\dev\team-eval\match_db.db")
df = pd.read_sql_query("""SELECT * FROM btb_matches""", conn)

df

,division_id,division,season_id,season,match_date,match_teams,home_team_id,home_team,away_team_id,away_team,...,home_deep,away_deep,home_ppda,away_ppda,bet365_home_odds,bet365_draw_odds,bet365_away_odds,bet365_u25_odds,bet365_o25_odds,match_id
0,98b8784f6685b7289f583e0ce4b4f6f2,Premier League,3ac445d3cc1d404987efdfcfa42f3bcd,20242025,2024-08-16 00:00:00,Man United - Fulham,f2b82cdbdadf9d3ec47c3a6be66dcfad,Man United,8cd5e94668b139c1f42a89a1e130f3cf,Fulham,...,7.0,3.0,7.379310,10.833333,1.60,4.20,5.25,2.50,1.53,Man United - Fulham_20240816
1,98b8784f6685b7289f583e0ce4b4f6f2,Premier League,3ac445d3cc1d404987efdfcfa42f3bcd,20242025,2024-08-17 00:00:00,Newcastle - Southampton,78e9266876e7649e0a12e3840f5be006,Newcastle,5a884401673693b0bdf379fefb7ec2b2,Southampton,...,4.0,13.0,16.250000,3.789474,1.36,5.25,8.00,3.00,1.40,Newcastle - Southampton_20240817
2,98b8784f6685b7289f583e0ce4b4f6f2,Premier League,3ac445d3cc1d404987efdfcfa42f3bcd,20242025,2024-08-17 00:00:00,Arsenal - Wolves,4fc9baf210346939946d5a49f255588b,Arsenal,31f3ecbc5c48590ccc7dabaedd49a4ff,Wolves,...,14.0,2.0,7.769231,10.818182,1.18,7.50,13.00,2.75,1.44,Arsenal - Wolves_20240817
3,98b8784f6685b7289f583e0ce4b4f6f2,Premier League,3ac445d3cc1d404987efdfcfa42f3bcd,20242025,2024-08-17 00:00:00,Nott'm Forest - Bournemouth,9a8e1e9fad8766fc3d69a0c26d98b928,Nott'm Forest,b436d55f36cfbe8a085c8b75fb7fe98a,Bournemouth,...,10.0,4.0,8.653846,9.954545,2.45,3.50,2.80,2.10,1.73,Nott'm Forest - Bournemouth_20240817
4,98b8784f6685b7289f583e0ce4b4f6f2,Premier League,3ac445d3cc1d404987efdfcfa42f3bcd,20242025,2024-08-17 00:00:00,Ipswich - Liverpool,e4f63bf6d6d2cd121e6c8e59bef68209,Ipswich,afce84ff226407a47c9782a742ba02f7,Liverpool,...,2.0,13.0,18.777778,8.739130,8.50,5.50,1.33,3.00,1.40,Ipswich - Liverpool_20240817
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8838,ef393faadcf7d00912ef326a1cd0f103,Ligue 1,06b84148ed1c6a06340478bef065f5bc,20212022,2022-05-21 00:00:00,Marseille - Strasbourg,8377de0f1845174610bd8b815b9a285e,Marseille,c651bda18ca0a5704727ed63d2bd3e8b,Strasbourg,...,9.0,5.0,9.850000,13.860000,2.05,3.60,3.60,2.10,1.72,Marseille - Strasbourg_20220521
8839,ef393faadcf7d00912ef326a1cd0f103,Ligue 1,06b84148ed1c6a06340478bef065f5bc,20212022,2022-05-21 00:00:00,Paris SG - Metz,541dd69b8e5c6a19f50d3dbf3819addf,Paris SG,a71f82f01730198a712591ceabb74238,Metz,...,14.0,2.0,8.560000,18.400000,1.20,8.50,10.00,4.33,1.22,Paris SG - Metz_20220521
8840,ef393faadcf7d00912ef326a1cd0f103,Ligue 1,06b84148ed1c6a06340478bef065f5bc,20212022,2022-05-21 00:00:00,Angers - Montpellier,eeaba7c0c0cd90d3286ae34bf3131fc1,Angers,375c7d274d5b84410e7b2bf0a00a9465,Montpellier,...,6.0,4.0,8.340000,16.470000,2.00,3.75,3.50,2.30,1.61,Angers - Montpellier_20220521
8841,ef393faadcf7d00912ef326a1cd0f103,Ligue 1,06b84148ed1c6a06340478bef065f5bc,20212022,2022-05-21 00:00:00,Lorient - Troyes,d14a6b82c2ad05082f0d7bf372b9994f,Lorient,bd3ff00cf2a68c244fdbb4fb0e513bc8,Troyes,...,8.0,4.0,8.190000,15.810000,2.15,3.50,3.40,2.10,1.72,Lorient - Troyes_20220521


In [6]:
final_df = pd.read_sql_query("""SELECT * FROM xgb_match_features_0065_001""", conn)

final_df[final_df["match_date"] > '2024-08-01'].to_csv("test_data.csv")

In [7]:
final_df["match_date"] = pd.to_datetime(final_df["match_date"])

final_df = final_df[final_df["match_date"] > '2021-12-31']

#final_df = final_df[final_df["division"] == "Premier League"]
final_df["xg_interaction"] = final_df["rolling_xg_for"] * final_df["opponent_rolling_xg_against"]
final_df["rolling_finishing_ability"] = final_df["rolling_goals_for"] - final_df["rolling_xg_for"]
final_df["rolling_opponent_finishing_ability"] = final_df["opponent_rolling_goals_against"] - final_df["opponent_rolling_xg_against"]

feature_cols = ["division", "home?", "avg_market_value", "opponent_avg_market_value", "rolling_finishing_ability", "rolling_opponent_finishing_ability", #"team_season", "opponent_team_season",#"xg_interaction",
             "rolling_goals_for", "rolling_xg_for", "rolling_shots_for", "rolling_shots_on_target_for", "rolling_corners_for", "rolling_deep_for", "rolling_ppda_for", 'rolling_goals_for_30d', 'rolling_goals_against_30d', 'rolling_xg_for_30d', 'rolling_xg_against_30d', 'rolling_shots_for_30d', 'rolling_shots_against_30d', 'rolling_shots_on_target_for_30d', 'rolling_shots_on_target_against_30d', 'rolling_corners_for_30d', 'rolling_corners_against_30d', 'rolling_deep_for_30d', 'rolling_deep_against_30d', 'rolling_ppda_for_30d', 'rolling_ppda_against_30d', 
             "opponent_rolling_goals_against", "opponent_rolling_xg_against", "opponent_rolling_shots_against", "opponent_rolling_shots_on_target_against", "opponent_rolling_corners_against", "opponent_rolling_deep_against", "opponent_rolling_ppda_against", 'opponent_rolling_goals_for_30d', 'opponent_rolling_goals_against_30d', 'opponent_rolling_xg_for_30d', 'opponent_rolling_xg_against_30d', 'opponent_rolling_shots_for_30d', 'opponent_rolling_shots_against_30d', 'opponent_rolling_shots_on_target_for_30d', 'opponent_rolling_shots_on_target_against_30d', 'opponent_rolling_corners_for_30d', 'opponent_rolling_corners_against_30d', 'opponent_rolling_deep_for_30d', 'opponent_rolling_deep_against_30d', 'opponent_rolling_ppda_for_30d', 'opponent_rolling_ppda_against_30d']

X = final_df[feature_cols]

# Convert categorical columns to category type
cat_cols = ["division"] #"team", "opponent_team"
for col in cat_cols:
    X[col] = X[col].astype('category')


y = final_df["goals"] 

text_cutoff = pd.to_datetime('2024-08-01')
X_train = X[final_df['match_date'] < text_cutoff]
y_train = y[final_df['match_date'] < text_cutoff]
X_test = X[final_df['match_date'] >= text_cutoff]
y_test = y[final_df['match_date'] >= text_cutoff]

print(f"Training set size: {X_train.shape[0]} samples")
print(f"Test set size: {X_test.shape[0]} samples")

model_params = {
        'n_estimators': 100,
        'learning_rate': 0.03,
        'max_depth': 5,
        'subsample': 0.7,
        'colsample_bytree': 0.9,
        'min_child_weight': 3,
        'gamma': 0.1,
        'alpha': 1,
        #'lambda_': 1,
        'random_state': 26,
        #'enable_categorical': True
    }

# Train an XGBoost model
model = xgb.XGBRegressor(objective='count:poisson', **model_params, enable_categorical= True)

model.fit(X_train, y_train)

# Make predictions
y_train_pred = model.predict(X_train)
y_pred = model.predict(X_test)

# Evaluate model
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)
train_mae = mean_absolute_error(y_train, y_train_pred)
print(f"RMSE: {rmse:.4f}")
print(f"Test MAE: {mae:.4f}")
print(f"Train MAE: {train_mae:.4f}")

X_test_has_nan = X_test.isna().any(axis=1)
y_test_has_nan = y_test.isna()

print(f"X_test rows with NaN: {X_test_has_nan.sum()}")
print(f"y_test rows with NaN: {y_test_has_nan.sum()}")

C:\Users\Owner\AppData\Local\Temp\ipykernel_2584\925527164.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = X[col].astype('category')


Training set size: 11772 samples
Test set size: 3562 samples
RMSE: 1.1462
Test MAE: 0.8994
Train MAE: 0.8709
X_test rows with NaN: 924
y_test rows with NaN: 0


In [8]:
# Get the test set indexes
test_indices = final_df.index[final_df['match_date'] >= text_cutoff]

# Create a temporary dataframe with all the test data
temp_df = final_df.loc[test_indices].copy()

# Initialize the predictions dataframe
match_preds = []

# Group by match_id to combine home and away rows
for match_id, group in temp_df.groupby('match_id'):
    if len(group) != 2:
        print(f"Warning: Match ID {match_id} does not have exactly 2 rows (has {len(group)})")
        continue
        
    # Get home and away rows
    home_row = group[group['home?'] == 1]
    away_row = group[group['home?'] == 0]
    
    if len(home_row) != 1 or len(away_row) != 1:
        print(f"Warning: Match ID {match_id} doesn't have one home and one away team")
        continue
    
    # Get the model predictions for both teams
    home_idx = home_row.index[0]
    away_idx = away_row.index[0]
    
    home_pred = y_pred[np.where(test_indices == home_idx)[0][0]] if home_idx in test_indices else None
    away_pred = y_pred[np.where(test_indices == away_idx)[0][0]] if away_idx in test_indices else None
    
    # Create a single row for this match
    match_data = {
        'match_id': match_id,
        'match_date': home_row['match_date'].values[0],
        'home_team': home_row['team'].values[0],
        'away_team': away_row['team'].values[0],
        'division': home_row['division'].values[0],
        'actual_home_goals': home_row['goals'].values[0],
        'actual_away_goals': away_row['goals'].values[0],
        'predicted_home_goals': home_pred,
        'predicted_away_goals': away_pred,
        'home_error': home_row['goals'].values[0] - home_pred if home_pred is not None else None,
        'away_error': away_row['goals'].values[0] - away_pred if away_pred is not None else None
    }
    
    # Add match_teams if available
    if 'match_teams' in home_row.columns:
        match_data['match_teams'] = home_row['match_teams'].values[0]
    
    match_preds.append(match_data)

# Create the final predictions dataframe
preds_df = pd.DataFrame(match_preds)

# Add some calculated columns
preds_df['home_abs_error'] = np.abs(preds_df['home_error'])
preds_df['away_abs_error'] = np.abs(preds_df['away_error'])
preds_df['total_abs_error'] = preds_df['home_abs_error'] + preds_df['away_abs_error']

# Save to CSV
#preds_df.to_csv("match_predictions.csv", index=False)

# Display summary stats
print("\nPrediction Summary:")
print(f"Total matches predicted: {len(preds_df)}")
print(f"Home goals MAE: {preds_df['home_abs_error'].mean():.4f}")
print(f"Away goals MAE: {preds_df['away_abs_error'].mean():.4f}")
print(f"Overall MAE: {(preds_df['home_abs_error'].mean() + preds_df['away_abs_error'].mean())/2:.4f}")
print(f"Date range: {preds_df['match_date'].min().date()} to {preds_df['match_date'].max().date()}")

preds_df


Prediction Summary:
Total matches predicted: 1781
Home goals MAE: 0.9413
Away goals MAE: 0.8576
Overall MAE: 0.8994
Date range: 2024-08-09 to 2025-03-27


,match_id,match_date,home_team,away_team,division,actual_home_goals,actual_away_goals,predicted_home_goals,predicted_away_goals,home_error,away_error,home_abs_error,away_abs_error,total_abs_error
0,Alaves - Ath Bilbao_20241215,2024-12-15,Alaves,Ath Bilbao,La Liga,1,1,1.066185,1.385801,-0.066185,-0.385801,0.066185,0.385801,0.451986
1,Alaves - Barcelona_20241006,2024-10-06,Alaves,Barcelona,La Liga,0,3,1.105241,1.716059,-1.105241,1.283941,1.105241,1.283941,2.389182
2,Alaves - Betis_20240825,2024-08-25,Alaves,Betis,La Liga,0,0,1.281096,1.275226,-1.281096,-1.275226,1.281096,1.275226,2.556322
3,Alaves - Celta_20250127,2025-01-27,Alaves,Celta,La Liga,1,1,1.164787,1.074873,-0.164787,-0.074873,0.164787,0.074873,0.239660
4,Alaves - Espanol_20250222,2025-02-22,Alaves,Espanol,La Liga,0,1,1.339459,0.968197,-1.339459,0.031803,1.339459,0.031803,1.371262
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1776,Wolves - Man City_20241020,2024-10-20,Wolves,Man City,Premier League,1,2,0.989141,2.207693,0.010859,-0.207693,0.010859,0.207693,0.218553
1777,Wolves - Man United_20241226,2024-12-26,Wolves,Man United,Premier League,2,0,1.227482,1.534383,0.772518,-1.534383,0.772518,1.534383,2.306902
1778,Wolves - Newcastle_20240915,2024-09-15,Wolves,Newcastle,Premier League,1,2,1.310014,1.813802,-0.310014,0.186198,0.310014,0.186198,0.496211
1779,Wolves - Nott'm Forest_20250106,2025-01-06,Wolves,Nott'm Forest,Premier League,0,3,1.186570,1.345101,-1.186570,1.654899,1.186570,1.654899,2.841469


In [9]:
def append_df_to_sqlite_table(df_new, db_path, table_name, key_columns=None, batch_size=500):
    # Make a copy to avoid modifying the original dataframe
    df = df_new.copy()
    
    # Connect to the database
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    
    # Check if table exists
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name=?", (table_name,))
    table_exists = cursor.fetchone() is not None
    
    if table_exists:
        # If key_columns not specified, use all columns
        if key_columns is None:
            key_columns = list(df.columns)
        
        # Get columns from existing table
        cursor.execute(f"PRAGMA table_info({table_name})")
        existing_columns = [row[1] for row in cursor.fetchall()]
        
        # Ensure all key columns exist in both dataframe and table
        key_columns = [col for col in key_columns if col in df.columns and col in existing_columns]
        
        if key_columns:
            # Process in batches to avoid SQL expression tree too large error
            filtered_dfs = []
            
            # Split dataframe into batches
            total_rows = len(df)
            for start_idx in range(0, total_rows, batch_size):
                end_idx = min(start_idx + batch_size, total_rows)
                batch_df = df.iloc[start_idx:end_idx]
                
                # For this batch, extract all key column values
                key_values = {}
                for key in key_columns:
                    key_values[key] = batch_df[key].tolist()
                
                # Build an optimized query - using IN clause for each key column
                where_clauses = []
                params = []
                
                for key in key_columns:
                    if len(key_values[key]) > 0:
                        placeholders = ','.join(['?'] * len(key_values[key]))
                        where_clauses.append(f"{key} IN ({placeholders})")
                        params.extend(key_values[key])
                
                if where_clauses:
                    # Get existing records
                    query = f"SELECT {', '.join(key_columns)} FROM {table_name} WHERE {' OR '.join(where_clauses)}"
                    cursor.execute(query, params)
                    existing_rows = cursor.fetchall()
                    
                    # Create a set of tuples representing existing rows
                    if key_columns:
                        existing_keys = set(existing_rows)
                        
                        # Filter out rows that already exist
                        batch_filtered = batch_df[~batch_df[key_columns].apply(tuple, axis=1).isin(existing_keys)]
                        if not batch_filtered.empty:
                            filtered_dfs.append(batch_filtered)
                else:
                    # No valid key values, include all rows
                    filtered_dfs.append(batch_df)
            
            # Combine all filtered batches
            if filtered_dfs:
                df_filtered = pd.concat(filtered_dfs, ignore_index=True)
                if not df_filtered.empty:
                    df_filtered.to_sql(table_name, conn, if_exists='append', index=False)
                    print(f"Added {len(df_filtered)} new records to table {table_name}")
                else:
                    print("No new records to append.")
            else:
                print("No new records to append.")
        else:
            # If no valid key columns, append all data (may create duplicates)
            df.to_sql(table_name, conn, if_exists='append', index=False)
            print(f"Warning: No valid key columns. Added all {len(df)} records to table {table_name}")
    else:
        # If table doesn't exist, create it with the new data
        df.to_sql(table_name, conn, index=False)
        print(f"Created new table {table_name} with {len(df)} records")
    
    # Close the connection
    conn.close()

append_df_to_sqlite_table(preds_df, 'match_db.db', 'xgb_pred_0065_001_prem20242025', key_columns=['match_id'])

Created new table xgb_pred_0065_001_prem20242025 with 1781 records


## Backtesting my model on 1x2 and o/u 2.5

In [10]:
backtest_df = pd.read_sql_query("""
    SELECT
        preds.match_id,
        preds.predicted_home_goals,
        preds.actual_home_goals,
        preds.predicted_away_goals,
        preds.actual_away_goals,
        btb.bet365_home_odds,
        btb.bet365_draw_odds,
        btb.bet365_away_odds,
        btb.bet365_u25_odds,
        btb.bet365_o25_odds
    FROM
        xgb_pred_0065_001_prem20242025 preds
    JOIN
        btb_matches btb
            ON btb.match_id = preds.match_id
                                
    """, conn)

backtest_df

,match_id,predicted_home_goals,actual_home_goals,predicted_away_goals,actual_away_goals,bet365_home_odds,bet365_draw_odds,bet365_away_odds,bet365_u25_odds,bet365_o25_odds
0,Alaves - Ath Bilbao_20241215,1.066185,1,1.385801,1,3.60,3.10,2.15,1.53,2.50
1,Alaves - Barcelona_20241006,1.105241,0,1.716059,3,5.25,4.50,1.57,2.30,1.62
2,Alaves - Betis_20240825,1.281096,0,1.275226,0,2.38,3.10,3.25,1.57,2.38
3,Alaves - Celta_20250127,1.164787,1,1.074873,1,2.63,3.00,3.00,1.67,2.20
4,Alaves - Espanol_20250222,1.339459,0,0.968197,1,1.80,3.25,5.25,1.50,2.63
...,...,...,...,...,...,...,...,...,...,...
1776,Wolves - Man City_20241020,0.989141,1,2.207693,2,8.00,6.00,1.33,2.75,1.44
1777,Wolves - Man United_20241226,1.227482,2,1.534383,0,3.90,3.70,1.91,2.20,1.67
1778,Wolves - Newcastle_20240915,1.310014,1,1.813802,2,3.40,3.50,2.10,2.38,1.57
1779,Wolves - Nott'm Forest_20250106,1.186570,0,1.345101,3,3.30,3.20,2.30,1.73,2.10


In [11]:
def simulate_match_outcomes(home_goals, away_goals, num_simulations=10000):

    # Generate random samples from Poisson distributions
    home_scores = np.random.poisson(home_goals, num_simulations)
    away_scores = np.random.poisson(away_goals, num_simulations)
    
    # Calculate total goals for each simulation
    total_goals = home_scores + away_scores
    
    # Count outcomes
    home_wins = np.sum(home_scores > away_scores)
    draws = np.sum(home_scores == away_scores)
    away_wins = np.sum(home_scores < away_scores)
    
    # Count over/under 2.5 goals
    over_2_5 = np.sum(total_goals > 2.5)
    under_2_5 = np.sum(total_goals <= 2.5)
    
    # Convert to probabilities
    home_win_prob = home_wins / num_simulations
    draw_prob = draws / num_simulations
    away_win_prob = away_wins / num_simulations
    over_2_5_prob = over_2_5 / num_simulations
    under_2_5_prob = under_2_5 / num_simulations
    
    return {
        'home_win': home_win_prob,
        'draw': draw_prob,
        'away_win': away_win_prob,
        'over_2_5': over_2_5_prob,
        'under_2_5': under_2_5_prob
    }

# Add these probabilities to your prediction dataframe
backtest_df['probs'] = backtest_df.apply(
    lambda row: simulate_match_outcomes(row['predicted_home_goals'], row['predicted_away_goals']), 
    axis=1
)

# Extract individual probabilities to separate columns
backtest_df['home_win_prob'] = backtest_df['probs'].apply(lambda x: x['home_win'])
backtest_df['draw_prob'] = backtest_df['probs'].apply(lambda x: x['draw'])
backtest_df['away_win_prob'] = backtest_df['probs'].apply(lambda x: x['away_win'])
backtest_df['over_2_5_prob'] = backtest_df['probs'].apply(lambda x: x['over_2_5'])
backtest_df['under_2_5_prob'] = backtest_df['probs'].apply(lambda x: x['under_2_5'])

# Drop the intermediate dictionary column
backtest_df = backtest_df.drop('probs', axis=1)

append_df_to_sqlite_table(backtest_df, 'match_db.db', 'xgb_backtest_0065_001_prem20242025', key_columns=['match_id'])

backtest_df

Created new table xgb_backtest_0065_001_prem20242025 with 1781 records


,match_id,predicted_home_goals,actual_home_goals,predicted_away_goals,actual_away_goals,bet365_home_odds,bet365_draw_odds,bet365_away_odds,bet365_u25_odds,bet365_o25_odds,home_win_prob,draw_prob,away_win_prob,over_2_5_prob,under_2_5_prob
0,Alaves - Ath Bilbao_20241215,1.066185,1,1.385801,1,3.60,3.10,2.15,1.53,2.50,0.2883,0.2631,0.4486,0.4422,0.5578
1,Alaves - Barcelona_20241006,1.105241,0,1.716059,3,5.25,4.50,1.57,2.30,1.62,0.2446,0.2377,0.5177,0.5304,0.4696
2,Alaves - Betis_20240825,1.281096,0,1.275226,0,2.38,3.10,3.25,1.57,2.38,0.3582,0.2710,0.3708,0.4624,0.5376
3,Alaves - Celta_20250127,1.164787,1,1.074873,1,2.63,3.00,3.00,1.67,2.20,0.3758,0.2903,0.3339,0.3834,0.6166
4,Alaves - Espanol_20250222,1.339459,0,0.968197,1,1.80,3.25,5.25,1.50,2.63,0.4420,0.2842,0.2738,0.4015,0.5985
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1776,Wolves - Man City_20241020,0.989141,1,2.207693,2,8.00,6.00,1.33,2.75,1.44,0.1583,0.1944,0.6473,0.6124,0.3876
1777,Wolves - Man United_20241226,1.227482,2,1.534383,0,3.90,3.70,1.91,2.20,1.67,0.3050,0.2597,0.4353,0.5117,0.4883
1778,Wolves - Newcastle_20240915,1.310014,1,1.813802,2,3.40,3.50,2.10,2.38,1.57,0.2858,0.2265,0.4877,0.6007,0.3993
1779,Wolves - Nott'm Forest_20250106,1.186570,0,1.345101,3,3.30,3.20,2.30,1.73,2.10,0.3280,0.2615,0.4105,0.4663,0.5337


In [12]:
def kelly_expected(probability, decimal_odds, fraction = 1.0):
    if decimal_odds <= 1 or probability <= 0 or probability >= 1:
        return 0, 0
        
    # Calculate Kelly Criterion
    q = 1 - probability  # probability of losing
    kelly = (probability * (decimal_odds - 1) - q) / (decimal_odds - 1) # win rate * potential profit subtracitng prob of losing. Divided by potentail profit
    kelly = kelly * fraction  # Apply fractional Kelly
    kelly = max(0, kelly)  # No negative bets
    
    # Calculate Expected Value
    ev = (probability * (decimal_odds - 1)) - (1 - probability)
    ev = ev  # Convert to percentage
    
    return kelly, ev

prob = 0.6398
odds = 1.67
fraction = 0.25

bet_size, ev = kelly_expected(prob, odds, fraction)
print(f"Optimal bet size: {bet_size:.1%}")
print(f"Expected Value: {ev:.1%}")


Optimal bet size: 2.6%
Expected Value: 6.8%


In [34]:
home_win_df = pd.read_sql_query("""SELECT match_id, actual_home_goals, actual_away_goals, home_win_prob, bet365_home_odds FROM xgb_backtest_0065_001_prem20242025""", conn)

home_win_df["home_win?"] = (home_win_df["actual_home_goals"] > home_win_df["actual_away_goals"]).astype(int)

results = home_win_df.apply(
    lambda row: kelly_expected(row['home_win_prob'], row['bet365_home_odds'], fraction=0.25),
    axis=1
)

# Extract Kelly and EV values into separate columns
home_win_df['kelly_stake'] = results.apply(lambda x: x[0])
home_win_df['ev_percent'] = results.apply(lambda x: x[1])
home_win_df['is_value_bet'] = (home_win_df['ev_percent'] > 0.00) 
#print(home_win_df)

home_win_df['profit'] = home_win_df.apply(
    lambda row: row['kelly_stake'] * (row['bet365_home_odds'] - 1) if row['is_value_bet'] and row['home_win?'] == 1 
              else (-row['kelly_stake'] if row['is_value_bet'] else 0),  # Lose Kelly stake for lost bets
    axis=1
)

# Display summary statistics
value_bets = home_win_df[home_win_df['is_value_bet']]
print(f"Total matches analyzed: {len(home_win_df)}")
print(f"Value bets found: {len(value_bets)} ({len(value_bets)/len(home_win_df):.1%} of total)")

if len(value_bets) > 0:
    # Calculate performance of value bets
    won_bets = value_bets['home_win?'].sum()
    profit = value_bets['profit'].sum()
    
    print(f"Value bets won: {won_bets}/{len(value_bets)} ({won_bets/len(value_bets):.1%})")
    print(f"Total profit: {profit:.2f} units")
    print(f"ROI: {profit/len(value_bets)*100:.2f}%")

Total matches analyzed: 1781
Value bets found: 516 (29.0% of total)
Value bets won: 123/516 (23.8%)
Total profit: -1.83 units
ROI: -0.35%


In [32]:
away_win_df = pd.read_sql_query("""SELECT match_id, actual_home_goals, actual_away_goals, away_win_prob, bet365_away_odds FROM xgb_backtest_0065_001_prem20242025""", conn)

away_win_df["away_win?"] = (away_win_df["actual_away_goals"] > away_win_df["actual_home_goals"]).astype(int)

results = away_win_df.apply(
    lambda row: kelly_expected(row['away_win_prob'], row['bet365_away_odds'], fraction=0.25),
    axis=1
)

# Extract Kelly and EV values into separate columns
away_win_df['kelly_stake'] = results.apply(lambda x: x[0])
away_win_df['ev_percent'] = results.apply(lambda x: x[1])
away_win_df['is_value_bet'] = (away_win_df['ev_percent'] > 0.00)

away_win_df['profit'] = away_win_df.apply(
    lambda row: row['kelly_stake'] * (row['bet365_away_odds'] - 1) if row['is_value_bet'] and row['away_win?'] == 1 
              else (-row['kelly_stake'] if row['is_value_bet'] else 0),  # Lose Kelly stake for lost bets
    axis=1
)

# Display summary statistics
value_bets = away_win_df[away_win_df['is_value_bet']]
print(f"Total matches analyzed: {len(away_win_df)}")
print(f"Value bets found: {len(value_bets)} ({len(value_bets)/len(away_win_df):.1%} of total)")

if len(value_bets) > 0:
    # Calculate performance of value bets
    won_bets = value_bets['away_win?'].sum()
    profit = value_bets['profit'].sum()
    
    print(f"Value bets won: {won_bets}/{len(value_bets)} ({won_bets/len(value_bets):.1%})")
    print(f"Total profit: {profit:.2f} units")
    print(f"ROI: {profit/len(value_bets)*100:.2f}%")

Total matches analyzed: 1781
Value bets found: 1126 (63.2% of total)
Value bets won: 253/1126 (22.5%)
Total profit: -3.06 units
ROI: -0.27%


In [23]:
draw_df = pd.read_sql_query("""SELECT match_id, actual_home_goals, actual_away_goals, draw_prob, bet365_draw_odds FROM xgb_backtest_0065_001_prem20242025""", conn)

draw_df["draw?"] = (draw_df["actual_away_goals"] == draw_df["actual_home_goals"]).astype(int)

results = draw_df.apply(
    lambda row: kelly_expected(row['draw_prob'], row['bet365_draw_odds'], fraction=0.25),
    axis=1
)

# Extract Kelly and EV values into separate columns
draw_df['kelly_stake'] = results.apply(lambda x: x[0])
draw_df['ev_percent'] = results.apply(lambda x: x[1])
draw_df['is_value_bet'] = draw_df['ev_percent'] > 0.02

draw_df['profit'] = draw_df.apply(
    lambda row: row['kelly_stake'] * (row['bet365_draw_odds'] - 1) if row['is_value_bet'] and row['draw?'] == 1 
              else (-row['kelly_stake'] if row['is_value_bet'] else 0),  # Lose Kelly stake for lost bets
    axis=1
)

# Display summary statistics
value_bets = draw_df[draw_df['is_value_bet']]
print(f"Total matches analyzed: {len(draw_df)}")
print(f"Value bets found: {len(value_bets)} ({len(value_bets)/len(draw_df):.1%} of total)")

if len(value_bets) > 0:
    # Calculate performance of value bets
    won_bets = value_bets['draw?'].sum()
    profit = value_bets['profit'].sum()
    
    print(f"Value bets won: {won_bets}/{len(value_bets)} ({won_bets/len(value_bets):.1%})")
    print(f"Total profit: {profit:.2f} units")
    print(f"ROI: {profit/len(value_bets)*100:.2f}%")

Total matches analyzed: 1781
Value bets found: 299 (16.8% of total)
Value bets won: 58/299 (19.4%)
Total profit: -0.65 units
ROI: -0.22%


In [38]:
over_25_df = pd.read_sql_query("""SELECT match_id, actual_home_goals, actual_away_goals, over_2_5_prob, bet365_o25_odds FROM xgb_backtest_0065_001_prem20242025""", conn)

over_25_df["over_25?"] = (over_25_df["actual_away_goals"] + over_25_df["actual_home_goals"] > 2.5).astype(int)

results = over_25_df.apply(
    lambda row: kelly_expected(row['over_2_5_prob'], row['bet365_o25_odds'], fraction=0.25),
    axis=1
)

# Extract Kelly and EV values into separate columns
over_25_df['kelly_stake'] = results.apply(lambda x: x[0])
over_25_df['ev_percent'] = results.apply(lambda x: x[1])
over_25_df['is_value_bet'] = (over_25_df['ev_percent'] > 0.02) 

over_25_df['profit'] = over_25_df.apply(
    lambda row: row['kelly_stake'] * (row['bet365_o25_odds'] - 1) if row['is_value_bet'] and row['over_25?'] == 1 
              else (-row['kelly_stake'] if row['is_value_bet'] else 0),  # Lose Kelly stake for lost bets
    axis=1
)

# Display summary statistics
value_bets = over_25_df[over_25_df['is_value_bet']]
print(f"Total matches analyzed: {len(over_25_df)}")
print(f"Value bets found: {len(value_bets)} ({len(value_bets)/len(over_25_df):.1%} of total)")

if len(value_bets) > 0:
    # Calculate performance of value bets
    won_bets = value_bets['over_25?'].sum()
    profit = value_bets['profit'].sum()
    
    print(f"Value bets won: {won_bets}/{len(value_bets)} ({won_bets/len(value_bets):.1%})")
    print(f"Total profit: {profit:.2f} units")
    print(f"ROI: {profit/len(value_bets)*100:.2f}%")

Total matches analyzed: 1781
Value bets found: 454 (25.5% of total)
Value bets won: 189/454 (41.6%)
Total profit: -1.49 units
ROI: -0.33%


In [40]:
under_25_df = pd.read_sql_query("""SELECT match_id, actual_home_goals, actual_away_goals, under_2_5_prob, bet365_u25_odds FROM xgb_backtest_0065_001_prem20242025""", conn)

under_25_df["under_25?"] = (under_25_df["actual_away_goals"] + under_25_df["actual_home_goals"] <= 2.5).astype(int)

results = under_25_df.apply(
    lambda row: kelly_expected(row['under_2_5_prob'], row['bet365_u25_odds'], fraction=0.25),
    axis=1
)

# Extract Kelly and EV values into separate columns
under_25_df['kelly_stake'] = results.apply(lambda x: x[0])
under_25_df['ev_percent'] = results.apply(lambda x: x[1])
under_25_df['is_value_bet'] = under_25_df['ev_percent'] > 0.02

under_25_df['profit'] = under_25_df.apply(
    lambda row: row['kelly_stake'] * (row['bet365_u25_odds'] - 1) if row['is_value_bet'] and row['under_25?'] == 1 
              else (-row['kelly_stake'] if row['is_value_bet'] else 0),  # Lose Kelly stake for lost bets
    axis=1
)

# Display summary statistics
value_bets = under_25_df[under_25_df['is_value_bet']]
print(f"Total matches analyzed: {len(under_25_df)}")
print(f"Value bets found: {len(value_bets)} ({len(value_bets)/len(under_25_df):.1%} of total)")

if len(value_bets) > 0:
    # Calculate performance of value bets
    won_bets = value_bets['under_25?'].sum()
    profit = value_bets['profit'].sum()
    
    print(f"Value bets won: {won_bets}/{len(value_bets)} ({won_bets/len(value_bets):.1%})")
    print(f"Total profit: {profit:.2f} units")
    print(f"ROI: {profit/len(value_bets)*100:.2f}%")

Total matches analyzed: 1781
Value bets found: 419 (23.5% of total)
Value bets won: 160/419 (38.2%)
Total profit: -0.61 units
ROI: -0.15%
